In [1]:
import pandas as pd
import seaborn as sns

In [2]:
data = pd.read_csv("../Data/Amazon/full_dataset.csv")
data.head()

,name,main_category,sub_category
0,TRIPIN Brass Black and Silver Kurta Buttons wi...,accessories,Fashion & Silver Jewellery
1,Colorsole® Domestic RO Booster Pump Head for 7...,appliances,All Appliances
2,Nike Men's Rosherun Nm Br Lt Retro/White/Volt ...,stores,Sportswear
3,Ahen Black Thread Bracelet Stylish Adjustable ...,accessories,Jewellery
4,imee 60cm (24 inch) Premium Series Normal LED ...,"tv, audio & cameras",Televisions


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103170 entries, 0 to 1103169
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   name           1103170 non-null  object
 1   main_category  1103170 non-null  object
 2   sub_category   1103170 non-null  object
dtypes: object(3)
memory usage: 25.2+ MB


In [4]:
data.nunique()

name             396210
main_category        20
sub_category        112
dtype: int64

In [5]:
data.duplicated().sum()

np.int64(642778)

In [6]:
data[data.duplicated()]

,name,main_category,sub_category
398,Malabar Gold & Diamonds 22k (916) Yellow Gold ...,accessories,Gold & Diamond Jewellery
437,PC Jeweller 22k (916) Yellow Gold Ring for Women,accessories,Gold & Diamond Jewellery
552,Avsar 18k (750) Yellow Gold and Diamond Ring f...,accessories,Gold & Diamond Jewellery
706,Candere by Kalyan Jewellers 18k (750) 22k (916...,accessories,Gold & Diamond Jewellery
751,Zeya Yellow Gold Ring,accessories,Gold & Diamond Jewellery
...,...,...,...
1103165,"Nishi® Waterproof Backpack, Girls & Women Styl...",accessories,Bags & Luggage
1103166,"FEROC XEGA Rubber Cricket Shoes (8, Blue)",men's shoes,Sports Shoes
1103167,Sasitrends Rhodium Silver Plated Leaf Shaped A...,accessories,Fashion & Silver Jewellery
1103168,Duke Stardust Men Slim Fit Stretchable Jeans,men's clothing,Jeans


In [7]:
data.drop_duplicates(inplace = True)

In [8]:
data["main_category"].value_counts()

main_category
accessories                92985
women's clothing           65854
tv, audio & cameras        63818
men's shoes                48808
men's clothing             44001
appliances                 31168
stores                     27405
home & kitchen             14139
kids' fashion              12254
sports & fitness           12179
bags & luggage              9990
beauty & health             9855
car & motorbike             7008
toys & baby products        5788
women's shoes               5095
industrial supplies         4093
grocery & gourmet foods     3256
pet supplies                1594
music                       1078
home, kitchen, pets           24
Name: count, dtype: int64

# Model 1 -> Using name(notes) and main category columns as Training and test data.
##### X value(independent variable) = name(notes)
##### Y value(dependent variable) = main category
##### We classify the categories based on their names. There are around 20 category values and around 1 million+ data tuples. We divide the data into the train set, the validation set, and the test set. And then we use metrics to evaluate the model.

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer


In [10]:
new_data = data.iloc[:,:2]
new_data.head()

,name,main_category
0,TRIPIN Brass Black and Silver Kurta Buttons wi...,accessories
1,Colorsole® Domestic RO Booster Pump Head for 7...,appliances
2,Nike Men's Rosherun Nm Br Lt Retro/White/Volt ...,stores
3,Ahen Black Thread Bracelet Stylish Adjustable ...,accessories
4,imee 60cm (24 inch) Premium Series Normal LED ...,"tv, audio & cameras"


In [11]:
train_set_x,test_set_x,train_set_y,test_set_y = train_test_split(new_data["name"],new_data["main_category"],test_size = 0.2, random_state=42)

In [12]:
train_set_x.shape,test_set_x.shape,train_set_y.shape,test_set_y.shape

((368313,), (92079,), (368313,), (92079,))

In [13]:
x_train,x_val,y_train,y_val = train_test_split(train_set_x,train_set_y,test_size=0.3, random_state=42)

In [14]:
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((257819,), (110494,), (257819,), (110494,))

In [26]:
tfidf = TfidfVectorizer(max_features=200000, ngram_range=(1,3), sublinear_tf=True, min_df=3, analyzer="word")
x_train_vect = tfidf.fit_transform(x_train)
x_val_vect = tfidf.transform(x_val)

In [27]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier(
    loss="hinge",
    learning_rate= "optimal",
    eta0=0.1,
    penalty = "elasticnet",
    l1_ratio=0.25,
    alpha = 1e-5,
    validation_fraction=0.2,
    early_stopping=True,
    class_weight="balanced",
    fit_intercept=True,
    tol=1e-3,
    shuffle=True,
    max_iter=20000,
    warm_start=False,
    random_state=42,
    n_jobs=-1)

In [28]:
from sklearn.pipeline import make_pipeline
clf = make_pipeline(tfidf,sgd)

In [31]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
skf = StratifiedKFold(n_splits=7, random_state=42, shuffle=True)
cvs = cross_val_score(cv=skf,estimator=clf, scoring="accuracy",n_jobs=-1,X=x_train,y=y_train,verbose=False)
print("CV Accuracy:", cvs.mean(), cvs)

Exception ignored in: <function ResourceTracker.__del__ at 0x107865bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes


CV Accuracy: 0.8726470835881079 [0.87315378 0.87369679 0.87130406 0.87149412 0.87437213 0.87149412
 0.87301458]


Exception ignored in: <function ResourceTracker.__del__ at 0x102a59bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x1075e9bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x104e7dbc0>
Traceback (most recent call last

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
y_val_pred = sgd.predict(x_val_vect)
print(accuracy_score(y_val,y_val_pred))
print(classification_report(y_val, y_val_pred))

In [ ]:
test_set_x_vect = tfidf.transform(test_set_x)
y_test_pred = svm.predict(test_set_x_vect)

In [ ]:
print(accuracy_score(test_set_y,y_test_pred))
print(classification_report(test_set_y, y_test_pred))

In [ ]:
cm = confusion_matrix(test_set_y, y_test_pred)
sns.heatmap(cm, cmap="Blues")

In [ ]:
import joblib
joblib.dump(svm, "svc_model.pkl")

In [ ]:
joblib.dump(tfidf,"tfidf_vectorizer.pkl")

In [ ]:
load_svm = joblib.load("svc_model.pkl")
load_vect = joblib.load("tfidf_vectorizer.pkl")

text = ["My t-shirt looks very cool, and trousers are really awesome"]
tranformed_text = load_vect.transform(text)

In [ ]:
load_svm.predict(tranformed_text)

In [ ]:
import spark